# CNN

## 5.1 2D Conv
### 5.1.1 Conv Operation

In [1]:
from mxnet import autograd, nd
from mxnet.gluon import nn

/Users/i058959/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def corr2d(X: nd.array, K: nd.array):
    """
    [2D convolution operaton for input X via Kernel K. Assumption: pad=1, stride=1.]
    
    Arguments:
        X {[nd.array]} -- [input data in matrix format]
        K {[nd.array]} -- [kernel matrix]
    
    Returns:
        [Y] -- [result after convolution]
    """
    h, w = K.shape
    Y = nd.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i+h, j: j+w] * K).sum()
    return Y

In [3]:
X = nd.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = nd.array([[0, 1], [2, 3]])
corr2d(X, K)


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @cpu(0)>

### 5.1.2 2D Conv Layer

In [4]:
class Conv2D(nn.Block):
    def __init__(self, kernel_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=kernel_size)
        self.bias = sle.params.get('bias', shape=(1,))
    
    def forward(self, x):
        return corr2d(x, self.weight.data()) + self.bias.data()

### 5.1.3 Boundary Detection

In [5]:
X = nd.ones((6, 8))
X[:, 2:6] = 0
# create image 0 with clear boundary
X


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

In [6]:
# we design kernel
K = nd.array([[1, -1]])

In [7]:
# detection of 0->1->-1->0
Y = corr2d(X, K)
Y


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

### 5.1.4 Learn Kernel Array Via Data

Using MSE to learn Kernel Array
Page 140

In [8]:
# construct output channel number = 1, kernel shape = (1, 2)'s 2D convolution layer
conv2d = nn.Conv2D(1, kernel_size=(1, 2))
conv2d.initialize()

# 2D convolution layer with 4 dimension inputs: (batch_size, channel number, height, width), here batch_size and chanel number = 1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    with autograd.record():
        Y_hat = conv2d(X)
        l = (Y_hat - Y) ** 2
        if i % 2 == 1:
            print("batch %d, loss %.3f" % (i, l.sum().asscalar()))
    l.backward()
    ## Issue: src/imperative/imperative.cc:193: Check failed: AGInfo::IsNone(*(outputs[i])) Assigning to NDArrays that are already in a computational graph will cause undefined behavior when evaluating gradients. Please call backward first to clear the graph or do this out side of a record section. Also note that you cannot use inplace operations like +=, *=, relu(x, out=x), y[idx]=x, etc inside a record section.            
    # if tab-forward for 'conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()', then it will be the scope of 'with autograd.record():', l.backward() won't work either
    # then you will get issue : as autograd.record() will keep value stable, however, weight.data is changing...
    # for demo purpose, just ignoring bias
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()

batch 1, loss 4.949
batch 3, loss 0.831
batch 5, loss 0.140
batch 7, loss 0.024
batch 9, loss 0.004


In [9]:
# check learnt kernel after 10 iteration, which is close to our previous "K = nd.array([[1, -1]])"
conv2d.weight.data().reshape((1, 2))


[[ 0.9895    -0.9873705]]
<NDArray 1x2 @cpu(0)>

### Exercise for 5.1.4 Learn Kernel Array Via Data

Question 1: How to detect horzitonal edge by which Kernel? How to detect diagnoise edge by which Kernel?